In [20]:
import multiprocessing
import os
import subprocess
import sys
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
from azure.storage.blob import BlobServiceClient
from azure.cli.core import get_default_cli

def az_cli (args_str):
    args = args_str.split()
    cli = get_default_cli()
    cli.invoke(args, out_file = open(os.devnull, 'w'))
    if cli.result.result:
        return cli.result.result
    elif cli.result.error:
        raise cli.result.error
    return True

In [21]:
# Get Experimentname
if len(sys.argv) == 2:
    experimentname = sys.argv[1]
else:
    experimentname = input("Enter experiment name:")

Enter experiment name:gtbf


In [23]:
azCliSA = az_cli(f"storage account show-connection-string --name {experimentname}core")
download_path = f"./data/downloaded/{experimentname}"
Path(download_path).mkdir(parents=True, exist_ok=True)

In [24]:
blob_service = BlobServiceClient.from_connection_string(azCliSA['connectionString'])
container_client = blob_service.get_container_client("runs")
# List the blobs in the container:
blob_list = container_client.list_blobs()

# List the already downloaded elements:
downloaded_tars = os.listdir(download_path)
downloaded_tars

[]

In [25]:
def download(blob):
    if blob.name in downloaded_tars:
        return 0

    blob_client = blob_service.get_blob_client(container="runs", blob=blob)
    download_file_path = os.path.join(download_path, blob.name)

    with open(download_file_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())
    return 1

print('downloading')
result = []
with ThreadPoolExecutor(max_workers=multiprocessing.cpu_count()) as tpe:
    result = tpe.map(download, blob_list)

print("downloaded", sum(result))

downloading
downloaded 91524
